<table align="center">
  <a target="_blank" href="https://colab.research.google.com/github/martinlf6/schwab-ds-takehome-FengLiu/blob/main/03_models.ipynb">
        <img src="https://i.ibb.co/2P3SLwK/colab.png"  style="padding-bottom:5px;" />Run in Google Colab</a>
</table>

In [1]:
!pip install datasets==3.6.0 --force-reinstall


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
# Load dataset
ds = load_dataset("financial_phrasebank", "sentences_allagree") # 'sentences_allagree' means only sentences where all annotators agreed on the sentiment label are included in the loaded dataset. ds is now a DatasetDict object with splits like "train"
df = ds["train"].to_pandas().rename(columns={"sentence":"text","label":"y"}) # Convert to Pandas dataframe. ds["train"] selects the training split of the dataset
label_map = {0: "negative", 1: "neutral", 2: "positive"} # Create a mapping from numbers to labels
df["label"] = df["y"].map(label_map) # Apply the mapping: replaces each numeric value in column y with its text label and creates a new column label with human-readable sentiment.
df["len"] = df["text"].str.split().apply(len) # .split() splits each list in text column (sentence) on whitespace, another word, splits each sentence into words. apply(len) applies the built-in Python len() function to each list in text column (sentence) that gives the number of words in the sentence.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

financial_phrasebank.py: 0.00B [00:00, ?B/s]

The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


FinancialPhraseBank-v1.0.zip:   0%|          | 0.00/682k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

In [3]:
df

,text,y,label,len
0,"According to Gran , the company has no plans t...",1,neutral,25
1,"For the last quarter of 2010 , Componenta 's n...",2,positive,39
2,"In the third quarter of 2010 , net sales incre...",2,positive,29
3,Operating profit rose to EUR 13.1 mn from EUR ...,2,positive,24
4,"Operating profit totalled EUR 21.1 mn , up fro...",2,positive,22
...,...,...,...,...
2259,Operating result for the 12-month period decre...,0,negative,27
2260,HELSINKI Thomson Financial - Shares in Cargote...,0,negative,40
2261,LONDON MarketWatch -- Share prices ended lower...,0,negative,26
2262,Operating profit fell to EUR 35.4 mn from EUR ...,0,negative,23


In [6]:
# Train/validation split: splits data into 80% train, 20% validation, stratified by label (balanced) ensuring that the proportion of each class (or label) is maintained in both subsets.
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["text"].tolist(), df["y"].tolist(), test_size=0.2, random_state=42, stratify=df["y"]
)


In [7]:
# Tokenization
MODEL_NAME = "ProsusAI/finbert"   # Loads FinBERT, a BERT model pre-trained on financial texts.
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) # Load the Tokenizer that will convert raw text (words/sentences) into numeric IDs that the model understands: input_ids (integer IDs) representing tokens and attention_mask (1s and 0s) showing which tokens are real vs. padding.

# Tokenizes sentences into IDs the model can process.
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128) # Truncation and padding guarantee that every input has exactly 128 tokens.
val_encodings   = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [9]:
train_encodings

{'input_ids': [[101, 16239, 11404, 12941, 6063, 5253, 9725, 2010, 12832, 2006, 3259, 4115, 2595, 2013, 1036, 1036, 8699, 1005, 1005, 2000, 1036, 1036, 4965, 1005, 1005, 1999, 2089, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 9413, 6137, 15714, 1055, 1009, 4748, 2545, 16344, 1009, 2572, 2038, 3130, 2218, 2195, 4460, 1999, 5446, 1998, 2491, 2012, 22098, 6125, 2164, 3772, 2004, 1996, 2449, 2177, 11486, 1998, 2383, 1996, 5971, 11486, 2597, 2012, 22098, 22108, 6125, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
# Define aspect extraction function
# Aspects: organizations, products (e.g., “ETF” (Exchange-Traded Fund), “credit card”, “mobile app”), key financial facets (“revenue”, “guidance”, “fees”).)
def extract_aspects(text):
    doc = nlp(text) # Runs the input text through spaCy’s NLP pipeline that will produce a Doc object with tokens, entities, and noun chunks.
    aspects = [] # Initializes an empty list aspects to store extracted phrases.

    # Named entities of interest
    for ent in doc.ents: # Loops through recognized named entities (doc.ents).
        if ent.label_ in {"ORG","PRODUCT","WORK_OF_ART"}:  # Keeps only those labeled as: "ORG", "PRODUCT" (products such as iPhone and Coca-Cola), and "WORK_OF_ART" (titles, works, etc.). Extend if needed.
            aspects.append(ent.text) # Add the text to the aspects list.

    # Extract financial noun chunks for financial facets
    for nc in doc.noun_chunks: # Iterates over noun phrases (e.g., "the quarterly revenue", "profit margin")
        head = nc.root.lemma_.lower() # Gets the root word’s lemma (base form)
        if head in {"revenue","guidance","dividend","yield","fees","costs","margin","outlook","results","stock","shares"}: # If the root is one of the predefined financial terms (like "revenue", "margin", "stock", etc.), it’s considered an aspect.
            aspects.append(nc.text) # Adds the entire chunk (nc.text) to aspects.

    # Deduplicate results: returns a cleaned list of aspects.
    aspects = list(dict.fromkeys(a.strip() for a in aspects if len(a.strip())>1)) # a.strip(): removes leading/trailing whitespace. len(a.strip())>1: ignores meaningless single-character tokens. dict.fromkeys(...): removes duplicates while preserving order.
    return aspects

df["aspects"] = df["text"].apply(extract_aspects) # Creates a new column aspects containing a list of extracted aspects for each sentence.
df["num_aspects"] = df["aspects"].str.len() # Creates another column num_aspects with the number of aspects found.
print(df[["text","aspects"]].head(10))


                                                text  \
0  According to Gran , the company has no plans t...   
1  For the last quarter of 2010 , Componenta 's n...   
2  In the third quarter of 2010 , net sales incre...   
3  Operating profit rose to EUR 13.1 mn from EUR ...   
4  Operating profit totalled EUR 21.1 mn , up fro...   
5  Finnish Talentum reports its operating profit ...   
6  Clothing retail chain Sepp+ñl+ñ 's sales incre...   
7  Consolidated net sales increased 16 % to reach...   
8  Foundries division reports its sales increased...   
9  HELSINKI ( AFX ) - Shares closed higher , led ...   

                             aspects  
0                                 []  
1                       [Componenta]  
2                                 []  
3                                 []  
4                                 []  
5                 [Finnish Talentum]  
6                        [Sepp+ñl+ñ]  
7                                 []  
8          [Foundries, Machine S

In [ ]:
df

,text,y,label,len,aspects,num_aspects
0,"According to Gran , the company has no plans t...",1,neutral,25,[],0
1,"For the last quarter of 2010 , Componenta 's n...",2,positive,39,[Componenta],1
2,"In the third quarter of 2010 , net sales incre...",2,positive,29,[],0
3,Operating profit rose to EUR 13.1 mn from EUR ...,2,positive,24,[],0
4,"Operating profit totalled EUR 21.1 mn , up fro...",2,positive,22,[],0
...,...,...,...,...,...,...
2259,Operating result for the 12-month period decre...,0,negative,27,[],0
2260,HELSINKI Thomson Financial - Shares in Cargote...,0,negative,40,"[HELSINKI Thomson Financial, Cargotec]",2
2261,LONDON MarketWatch -- Share prices ended lower...,0,negative,26,"[MarketWatch, bank stocks]",2
2262,Operating profit fell to EUR 35.4 mn from EUR ...,0,negative,23,[],0


In [ ]:
# Transform the sentence-level dataset into an aspect-level dataset
pairs = []
for _, row in df.iterrows(): # Loops through the DataFrame df row by row.
    s = row["text"]; sent_label = row["label"]; aspects = row["aspects"] # s: the sentence text. sent_label: sentiment label (negative/neutral/positive). aspects: list of extracted aspects from the sentence.
    if not aspects: # Skip sentences with no aspects (the sentence with no aspects is not useful for aspect-level analysis)
        continue
    if len(aspects) == 1:
        pairs.append({"sentence": s, "aspect": aspects[0], "label": sent_label}) # If there’s exactly 1 aspect, create a dictionary with: "sentence" as full sentence text, "aspect" as the only aspect, and "label" as the sentence-level sentiment; and append it to pairs.
    else:
        # If there are multiple aspects in the same sentence: It duplicates the sentence into multiple rows, one for each aspect. Assigns the same sentence-level sentiment to each aspect (this is a simplification; in reality, some aspects might be positive while others are negative).
        for a in aspects:
            pairs.append({"sentence": s, "aspect": a, "label": sent_label})

pairs_df = pd.DataFrame(pairs) # Convert to DataFrame, and now each row = one aspect, instead of one sentence.
print(pairs_df.head())


                                            sentence            aspect  \
0  For the last quarter of 2010 , Componenta 's n...        Componenta   
1  Finnish Talentum reports its operating profit ...  Finnish Talentum   
2  Clothing retail chain Sepp+ñl+ñ 's sales incre...         Sepp+ñl+ñ   
3  Foundries division reports its sales increased...         Foundries   
4  Foundries division reports its sales increased...      Machine Shop   

      label  
0  positive  
1  positive  
2  positive  
3  positive  
4  positive  


In [ ]:
pairs_df

,sentence,aspect,label
0,"For the last quarter of 2010 , Componenta 's n...",Componenta,positive
1,Finnish Talentum reports its operating profit ...,Finnish Talentum,positive
2,Clothing retail chain Sepp+ñl+ñ 's sales incre...,Sepp+ñl+ñ,positive
3,Foundries division reports its sales increased...,Foundries,positive
4,Foundries division reports its sales increased...,Machine Shop,positive
...,...,...,...
2003,"Operating profits in the half were 0.8 m , do...",Glisten,negative
2004,HELSINKI Thomson Financial - Shares in Cargote...,HELSINKI Thomson Financial,negative
2005,HELSINKI Thomson Financial - Shares in Cargote...,Cargotec,negative
2006,LONDON MarketWatch -- Share prices ended lower...,MarketWatch,negative
